
<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 15px; height: 80px">

# Random Forests and ExtraTrees


_Authors: Matt Brems (DC), Riley Dallas (AUS), Patrick Wales-Dinan (SF)_

## Random Forests
---

With bagged decision trees, we generate many different trees on pretty similar data. These trees are **strongly correlated** with one another. Because these trees are correlated with one another, they will have high variance. Looking at the variance of the average of two random variables $T_1$ and $T_2$:

$$
\begin{eqnarray*}
Var\left(\frac{T_1+T_2}{2}\right) &=& \frac{1}{4}\left[Var(T_1) + Var(T_2) + 2Cov(T_1,T_2)\right]
\end{eqnarray*}
$$

If $T_1$ and $T_2$ are highly correlated, then the variance will be about as high as we'd see with individual decision trees. By "de-correlating" our trees from one another, we can drastically reduce the variance of our model.

That's the difference between bagged decision trees and random forests! We're going to do the same thing as before, but we're going to de-correlate our trees. This will reduce our variance (at the expense of a small increase in bias) and thus should greatly improve the overall performance of the final model.

So how do we "de-correlate" our trees?

Random forests differ from bagging decision trees in only one way: they use a modified tree learning algorithm that selects, at each split in the learning process, a **random subset of the features**. This process is sometimes called the *random subspace method*.

The reason for doing this is the correlation of the trees in an ordinary bootstrap sample: if one or a few features are very strong predictors for the response variable (target output), these features will be used in many/all of the bagged decision trees, causing them to become correlated. By selecting a random subset of features at each split, we counter this correlation between base trees, strengthening the overall model.

For a problem with $p$ features, it is typical to use:

- $\sqrt{p}$ (rounded down) features in each split for a classification problem.
- $p/3$ (rounded down) with a minimum node size of 5 as the default for a regression problem.

While this is a guideline, Hastie and Tibshirani (authors of Introduction to Statistical Learning and Elements of Statistical Learning) have suggested this as a good rule in the absence of some rationale to do something different.

Random forests, a step beyond bagged decision trees, are **very widely used** classifiers and regressors. They are relatively simple to use because they require very few parameters to set and they perform pretty well.
- It is quite common for interviewers to ask how a random forest is constructed or how it is superior to a single decision tree.

--- 

## Extremely Randomized Trees (ExtraTrees)
Adding one more step of randomization (and thus de-correlation) yields extremely randomized trees, or _ExtraTrees_. These are trained using bagging (sampling of observations) and the random subspace method (sampling of features), but an additional layer of randomness is introduced. Instead of computing the locally optimal feature/split combination (based on, e.g., information gain or the Gini impurity) for each feature under consideration, a random value is selected for the split. This value is selected from the feature's empirical range.

This further reduces the variance, but causes an increase in bias. If you're considering using ExtraTrees, you might consider this to be a hyperparameter you can tune. Build an ExtraTrees model and a Random Forest model, then compare their performance!

That's exactly what we'll do below.

## Import libraries
---

We'll need the following libraries for today's lecture:
- `pandas`
- `numpy`
- `GridSearchCV`, `train_test_split` and `cross_val_score` from `sklearn`'s `model_selection` module 
- `RandomForestClassifier` and `ExtraTreesClassifier` from `sklearn`'s `ensemble` module 

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV

## Load Titanic Data
---

Load `train.csv` and `test.csv` from Kaggle into `DataFrames`.

In [2]:
train = pd.read_csv('datasets/train.csv')

In [3]:
test = pd.read_csv('datasets/test.csv')

## Data Cleaning: Drop the two rows with missing `Embarked` values from train
---

In [4]:
train.dropna(subset = ['Embarked'], inplace = True)

In [5]:
train.shape

(889, 12)

## Data Cleaning: `Fare`
---

The test set has one row with a missing value for `Fare`. Fill it with the average `Fare` with everyone from the same `Pclass`. **Use the training set to calculate the average!**

In [6]:
test[test['Fare'].isnull()]

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
152,1044,3,"Storey, Mr. Thomas",male,60.5,0,0,3701,NaN,NaN,S


In [7]:
mean_fare_3 = train.loc[train['Pclass'] == 3, 'Fare'].mean()
mean_fare_3

13.675550101832997

In [8]:
test['Fare'].fillna(mean_fare_3, inplace = True)

In [9]:
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          327
Embarked         0
dtype: int64

## Data Cleaning: `Age`
---

Let's simply impute all missing ages to be **999**. 

**NOTE**: This is not a best practice. However, 
1. Since we haven't really covered imputation in depth
2. And the proper way would take too long to implement (thus detracting) from today's lecture
3. And since we're ensembling with Decision Trees

We'll do it this way as a matter of convenience.

In [10]:
train['Age'].fillna(999, inplace = True)

In [11]:
test['Age'].fillna(999, inplace = True)

## Feature Engineering: `Cabin`
---

Since there are so many missing values for `Cabin`, let's binarize that column as follows:
- 1 if there originally was a value for `Cabin`
- 0 if it was null

**Do this for both `train` and `test`**

In [12]:
train['Cabin'] = train['Cabin'].notnull().astype(int)

In [13]:
test['Cabin'] = test['Cabin'].notnull().astype(int)

## Feature Engineering: Dummies
---

Dummy the `Sex` and `Embarked` columns. Be sure to set `drop_first=True`.

In [14]:
train = pd.get_dummies(train, columns = ['Sex', 'Embarked'], drop_first = True)

In [15]:
test = pd.get_dummies(test, columns = ['Sex', 'Embarked'], drop_first = True)

## Model Prep: Create `X` and `y` variables
---

Our features will be:
- `Pclass`
- `Age`
- `SibSp`
- `Parch`
- `Fare`
- `Cabin`
- `Sex_male`
- `Embarked_Q`
- `Embarked_S`

And our target will be `Survived`

In [16]:
train['Survived'].value_counts()

0    549
1    340
Name: Survived, dtype: int64

In [17]:
features = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin', 'Sex_male', 'Embarked_Q', 'Embarked_S']

X = train[features]

y = train['Survived']

## Challenge: What is our baseline accuracy?
---

The baseline accuracy is the percentage of the majority class, regardless of whether it is 1 or 0. It serves as the benchmark for our model to beat.

In [18]:
# Sam's code

y.value_counts(normalize=True)

0    0.617548
1    0.382452
Name: Survived, dtype: float64

## Train/Test Split
---

I know it can be confusing having an `X_test` from our training data vs a test set from Kaggle. If you want, you can use `X_val`/`y_val` for what we normally call `X_test`/`y_test`.

In [19]:
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state = 42,
                                                 stratify = y)

## Model instantiation
---

Create an instance of `RandomForestClassifier` and `ExtraTreesClassifier`.

In [20]:
rf = RandomForestClassifier(n_estimators = 100)

In [21]:
et = ExtraTreesClassifier(n_estimators=100)

## Model Evaluation
---

Which one has a higher `cross_val_score`?

In [22]:
cross_val_score(rf, X_train, y_train, cv = 5).mean()

0.8033105150937043

In [23]:
cross_val_score(et, X_train, y_train, cv = 5).mean()

0.7778139378296488

## Grid Search
---

They're both pretty close performance-wise. We could Grid Search over both, but for the sake of time we'll go with `RandomForestClassifier`.

In [30]:
rf_params = {
    'n_estimators': [100, 150, 200],
    'max_depth': [None, 1, 2, 3, 4, 5],
    'random_state': [42]
}

gs = GridSearchCV(rf, param_grid=rf_params,
                 cv = 5,
                  n_jobs = -1)

gs.fit(X_train, y_train)

print(gs.best_score_)

gs.best_params_

0.8198406463920996


{'max_depth': 4, 'n_estimators': 200, 'random_state': 42}

In [31]:
gs.score(X_train, y_train)

0.8438438438438438

In [32]:
gs.score(X_val, y_val)

0.8116591928251121

## Kaggle Submission
---

Now that we've evaluated our model, let's submit our predictions to Kaggle.

In [33]:
final_model = gs.best_estimator_
final_model.fit(train[features], train['Survived'])

RandomForestClassifier(max_depth=4, n_estimators=200, random_state=42)

In [34]:
pred = final_model.predict(test[features])

In [35]:
test['Survived'] = pred

In [37]:
test[['PassengerId', 'Survived']].to_csv('submission_20211209.csv', index = False)

In [38]:
kaggle_test = pd.read_csv('submission_20211209.csv')

kaggle_test.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0


## On the Fly: RandomForestRegressor

[RF Regressor Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html)

In [39]:
from sklearn.ensemble import RandomForestRegressor

In [40]:
# read in the data

# Go to .csv file in GH, click on 'raw', paste link into pd.read_csv
df = pd.read_csv('https://raw.git.generalassemb.ly/DSIR-111/dsir-111/master/week-06/6.02-lesson-flask-heroku/model/data/ames.csv?token=AAAIYYDO4K7WSD5PEVCYLSTBXO3X2')

df.head()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
0,109,533352170,60,RL,NaN,13517,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,130500
1,544,531379050,60,RL,43.0,11492,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,4,2009,WD,220000
2,153,535304180,20,RL,68.0,7922,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,1,2010,WD,109000
3,318,916386060,60,RL,73.0,9802,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,4,2010,WD,174000
4,255,906425045,50,RL,82.0,14235,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,138500


In [41]:
good_cols = ['Overall Qual', 'Full Bath', 'Garage Area', 'Lot Area']

df.dropna(subset = good_cols, inplace = True)

X = df[good_cols]
y = df['SalePrice']

In [42]:
rf_reg = RandomForestRegressor()

In [43]:
rf_reg_params = {
    'n_estimators': [100, 150, 200],
    'max_depth': [None, 1, 2, 3, 4, 5],
    'random_state': [42]
}

gs = GridSearchCV(rf_reg, param_grid=rf_reg_params,
                 cv = 5,
                  n_jobs = -1)

gs.fit(X, y)

print(gs.best_score_)

gs.best_params_

0.7873709342902993


{'max_depth': 5, 'n_estimators': 200, 'random_state': 42}

In [45]:
rf_reg = gs.best_estimator_

In [46]:
rf_reg.feature_importances_

array([0.80734598, 0.03838076, 0.06308009, 0.09119317])